In [1]:
import pandas as pd
import src.maude_interface as maude
import logging

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
mdr_base = maude.maude_to_pandas("./data/mdrfoi.txt")
#foidev_test = maude.maude_to_pandas("./data/foidev.txt")

DEBUG:root:Reading file ./data/mdrfoi.txt


C:\Users\RISINGERAR1.2CE2211R26\Documents\GitHub\maude-analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
mdr_total = maude.add_data_to_mdr(mdr_base, ("./data/foidev1998.txt", "./data/foidevproblem.txt", "./data/deviceproblemcodes.txt"))

DEBUG:root:Reading file ./data/foidev1998.txt


DEBUG:root:Reading file ./data/foidevproblem.txt


C:\Users\RISINGERAR1.2CE2211R26\Documents\GitHub\maude-analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


DEBUG:root:Reading file ./data/deviceproblemcodes.txt


In [5]:
mdr_total = pd.merge(mdr_total, maude.maude_to_pandas("./reference/deviceproblemcodes.txt"), how='left', on='DEVICE_PROBLEM_CODE')

DEBUG:root:Reading file ./reference/deviceproblemcodes.txt


In [4]:
mdr_total.to_pickle("./data/mdrfoi_manual_compiled.pkl")
print(mdr_total.shape)
#print(mdr_total)

NameError: name 'mdr_total' is not defined

In [4]:
maude.compile_maude_database("./data/", "mdrfoiAdd.txt")

DEBUG:root:Constructing database from base file: ./data/mdrfoiAdd.txt


DEBUG:root:Reading file ./data/mdrfoiAdd.txt


C:\Users\RISINGERAR1.2CE2211R26\Documents\GitHub\maude-analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
DEBUG:root:Attempting to add file deviceproblemcodes.txt to dataset


DEBUG:root:Reading file ./data/deviceproblemcodes.txt


TypeError: merge() got multiple values for argument 'how'